In [44]:
from matplotlib import font_manager
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

font_path = '/System/Library/Fonts/Hiragino Sans GB.ttc'
my_font = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = my_font.get_name()
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示为方块的问题

# 读取数据
data = pd.read_csv('data/材料1.csv')

In [45]:
data = data[data['励磁波形'] == '正弦波']
print(data.shape)

(1067, 1028)


In [46]:
freq = data['频率，Hz']
loss = data['磁芯损耗，w/m3']

Bm_values = data.iloc[:, 4:].max(axis=1).values

In [47]:
def predict_loss(f, Bm, k1, alpha1, beta1):
    return k1 * (f ** alpha1) * (Bm ** beta1)

In [48]:
initial_guess = [1e-5, 1.5, 2.5]
# 用 curve_fit 来拟合参数，xdata 是足足一个二维数组。取出我们的 frequency 和 B_max 合并
popt, pcov = curve_fit(
    lambda xdata, k1, alpha1, beta1: predict_loss(xdata[0], xdata[1], k1, alpha1, beta1),
    (freq, Bm_values),  # 这里将 frequency 和 B_max 合并为 xdata
    loss, p0=initial_guess
)
k1, alpha1, beta1 = popt
print(f'拟合结果：k1={k1}, alpha1={alpha1}, beta1={beta1}')

拟合结果：k1=1.4997417251154614, alpha1=1.4296345737356402, beta1=2.4712537948884052


/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_22229/3652110823.py:2: RuntimeWarning: overflow encountered in power
  return k1 * (f ** alpha1) * (Bm ** beta1)


In [49]:
data.head()

,温度，oC,频率，Hz,磁芯损耗，w/m3,励磁波形,0（磁通密度B，T）,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,25,50030,1997.955250,正弦波,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001551,-0.001374,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036
1,25,50020,2427.749830,正弦波,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002476,-0.002284,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744
2,25,50020,3332.725760,正弦波,-0.003780,-0.003564,-0.003349,-0.003134,-0.002919,-0.002704,...,-0.005927,-0.005713,-0.005499,-0.005285,-0.005070,-0.004855,-0.004640,-0.004425,-0.004210,-0.003995
3,25,50020,4502.908007,正弦波,-0.000511,-0.000267,-0.000023,0.000222,0.000466,0.000711,...,-0.002950,-0.002707,-0.002463,-0.002220,-0.001976,-0.001732,-0.001488,-0.001244,-0.001000,-0.000756
4,25,50030,6063.023248,正弦波,0.000458,0.000732,0.001007,0.001281,0.001555,0.001830,...,-0.002290,-0.002016,-0.001741,-0.001466,-0.001192,-0.000917,-0.000642,-0.000367,-0.000092,0.000183


In [50]:
temperature_25 = data.loc[data['温度，oC'] == 25]
temperature_50 = data.loc[data['温度，oC'] == 50]
temperature_70 = data.loc[data['温度，oC'] == 70]
temperature_90 = data.loc[data['温度，oC'] == 90]
print(temperature_25.shape)
print(temperature_50.shape)
print(temperature_70.shape)
print(temperature_90.shape)

(271, 1028)
(256, 1028)
(265, 1028)
(275, 1028)


In [51]:
Bm_25 = temperature_25.iloc[:, 4:].max(axis=1).values
Bm_50 = temperature_50.iloc[:, 4:].max(axis=1).values
Bm_70 = temperature_70.iloc[:, 4:].max(axis=1).values
Bm_90 = temperature_90.iloc[:, 4:].max(axis=1).values

f_25 = temperature_25.iloc[:, 1].values
f_50 = temperature_50.iloc[:, 1].values
f_70 = temperature_70.iloc[:, 1].values
f_90 = temperature_90.iloc[:, 1].values

l_25 = temperature_25.iloc[:, 2].values
l_50 = temperature_50.iloc[:, 2].values
l_70 = temperature_70.iloc[:, 2].values
l_90 = temperature_90.iloc[:, 2].values



In [52]:
# mape = np.mean(np.abs((y_valid - y_pred) / y_valid)) * 100
T_25 = predict_loss(f_25, Bm_25, k1, alpha1, beta1)
T_50 = predict_loss(f_50, Bm_50, k1, alpha1, beta1)
T_70 = predict_loss(f_70, Bm_70, k1, alpha1, beta1)
T_90 = predict_loss(f_90, Bm_90, k1, alpha1, beta1)

mape_25 = np.mean(np.abs((l_25 - T_25) / l_25)) * 100
mape_50 = np.mean(np.abs((l_50 - T_50) / l_50)) * 100
mape_70 = np.mean(np.abs((l_70 - T_70) / l_70)) * 100
mape_90 = np.mean(np.abs((l_90 - T_90) / l_90)) * 100

In [53]:
print(f"温度 25 mape: {mape_25}")
print(f"温度 50 mape: {mape_50}")
print(f"温度 70 mape: {mape_70}")
print(f"温度 90 mape: {mape_90}")


温度 25 mape: 29.765093126110926
温度 50 mape: 10.691851800392126
温度 70 mape: 39.24048053100665
温度 90 mape: 61.272164953363784


In [54]:
from sklearn.metrics import mean_squared_error

mse_25 = mean_squared_error(l_25, T_25)
mse_50 = mean_squared_error(l_50, T_50)
mse_70 = mean_squared_error(l_70, T_70)
mse_90 = mean_squared_error(l_90, T_90)


In [55]:
print(f"温度 25 mse: {mse_25}")
print(f"温度 50 mse: {mse_50}")
print(f"温度 70 mse: {mse_70}")
print(f"温度 90 mse: {mse_90}")

温度 25 mse: 4149731982.870412
温度 50 mse: 67732136.29861777
温度 70 mse: 783333569.814949
温度 90 mse: 1448905610.8714879
